# Dusty wave

## Imports

In [ ]:
%load_ext blackcellmagic

In [ ]:
import pathlib

import numpy as np
import pandas as pd
import plonk

from bokeh.io import output_notebook, show
from bokeh.palettes import inferno, viridis
from bokeh.plotting import figure
output_notebook()

## Show an example simulation

In [ ]:
directory = pathlib.Path('~/runs/multigrain/dustywave.stretch/Epstein-f=10.0-MRN').expanduser()
sim = plonk.load_sim(prefix='dustywave', directory=directory)

# Select snaps to plot
snaps = sim.snaps[:10]

# Select particles to plot
s = slice(None, None, 1)

# Plot range
x_range = (-0.5, 0.5)

vx_max = np.abs(sim.snaps[0]['vx']).max() * 1.05
vx_range = (-vx_max, vx_max)

rho_mean = sim.snaps[0]['gas']['rho'].mean()
rho_var = 1e-4
rho_range = (rho_mean * (1 - rho_var), rho_mean * (1 + rho_var))

# Generate figures
fig1 = figure(x_range=x_range, y_range=vx_range)
fig2 = figure(x_range=x_range, y_range=rho_range)

for snap, color in zip(snaps, inferno(len(snaps))):
    fig1.scatter(
        snap["x"][s], snap["vx"][s], line_color=color, fill_color=color, size=5
    )
    fig2.scatter(
        snap["x"][s], snap["rho"][s], line_color=color, fill_color=color, size=5
    )
    
show(fig1)
show(fig2)

## Do the analysis

Path to data....

In [ ]:
root_directory = pathlib.Path('~/runs/multigrain/dustywave.stretch').expanduser()
paths = sorted(list(root_directory.glob('*')))

Calculate $\rho$ and $v_x$ at $x=0$ for each snapshot in each simulation. Each simulation has a data frame.

In [ ]:
A = 1e-4
c_s = 1

def do_analysis(sim):
    
    n_snaps = len(sim.snaps)
    n_species = len(sim.snaps[0].properties["grain size"]) + 1

    time = np.array([snap.properties["time"] for snap in sim.snaps])
    rho = np.zeros((n_snaps, n_species))
    vx = np.zeros((n_snaps, n_species))

    rho_0 = np.zeros(n_species)
    snap = sim.snaps[0]
    for dust_id in range(n_dust + 1):
        subsnap = snap[snap['dust_id'] == dust_id]
        rho_0[dust_id] = subsnap['rho'].mean()
        
    for idx, snap in enumerate(sim.snaps):
        mask = (np.abs(snap['x']) < 0.05)
        for dust_id in range(n_dust + 1):
            subsnap = snap[(snap["dust_id"] == dust_id) & mask]
            rho[idx, dust_id] = (subsnap["rho"].mean() - rho_0[dust_id]) / (A * rho_0[dust_id])
            vx[idx, dust_id] = subsnap["vx"].mean() / (A * c_s)

    arrays = np.hstack((time[:, np.newaxis], rho, vx))
    columns = (
        ["time"]
        + [f"rho.{idx}" for idx in range(n_species)]
        + [f"vx.{idx}" for idx in range(n_species)]
    )

    del sim
    
    return pd.DataFrame(arrays, columns=columns)

Loop over each simulation.

In [ ]:
dataframes = dict()

for p in paths:
    
    sim = plonk.load_sim(prefix="dustywave", directory=p)
    name = "-".join(sim.path.name.split("=")[-1].split("-")[::-1])
    print(f"Analysis for {name}")

    dataframes[name] = do_analysis(sim)

In [ ]:
x_range = 0.0, 10.0
y_range = -1, 1

p1 = figure(title='Normalized density', x_range=x_range, y_range=y_range)
p2 = figure(title='Normalized velocity', x_range=x_range, y_range=y_range)

colors = viridis(len(dataframes))

for df, color in zip(dataframes.values(), colors):
    
    x = np.array(df['time'])
    y = np.array(df['rho.0'])
    p1.line(x, y, line_color=color)
    p1.circle(x, y, line_color=color, fill_color=None)
    
    y = np.array(df['vx.0'])
    p2.line(x, y, line_color=color)
    p2.circle(x, y, line_color=color, fill_color=None)

show(p1)
show(p2)